In [1]:
import numpy as np
from numpy import testing

In [2]:
%run numerics.ipynb

In [1]:
class MPDATA:
    @staticmethod
    def _magn(q):
        return q.to_base_units().magnitude
    
    def __init__(self, nr, r_min, r_max, dt, cdf_r_lambda, coord, opts):
        self._nm = numerics()
        self.opts = opts
                
        self._n = 0

        #   |-----o-----|-----o--...
        # i-1/2   i   i+1/2   i+1
        # x_min     x_min+dx
        
        if opts["n_it"] > 1 and opts["dfl"]:
            n_halo = 2
        else:
            n_halo = 1
            
        self._i = slice(0, nr) + n_halo * self._nm.one
                
        # cell-border stuff
        self._ih = self._i % self._nm.hlf
        
        x_unit = coord.x(r_min).to_base_units().units
        
        _, self._dx = np.linspace(
            self._magn(coord.x(r_min)), 
            self._magn(coord.x(r_max)), 
            nr+1, 
            retstep=True
        )
        self._xh = np.linspace(
            self._magn(coord.x(r_min)) - (n_halo-1) * self._dx, 
            self._magn(coord.x(r_max)) + (n_halo-1) * self._dx, 
            nr+1 + 2*(n_halo - 1)
        )
        
        self._rh = coord.r(self._xh * x_unit)
        self._Gh = 1 / self._magn(coord.dx_dr(self._rh))
        self._GCh = np.full_like(self._Gh, np.nan)
        
        self._flux = np.full_like(self._Gh, np.nan)
        if opts["n_it"] > 1:
            self._GCh_tmp = np.full_like(self._Gh, np.nan)
        
        # cell-centered stuff
        self._x = np.linspace(
            self._xh[0] - self._dx/2,
            self._xh[-1] + self._dx/2, 
            nr + 2*n_halo 
        )
        self._r = coord.r(self._x * x_unit)

        self._G = np.full_like(self._x, np.nan)
        self._G = 1 / self._magn(coord.dx_dr(self._r))
        
        # dt
        self._dt = self._magn(dt)
        
        # psi from cdf
        self._psi = np.full_like(self._G, np.nan)
        self._psi = (self._psi, self._psi.copy())
        self._psi[-1][self._i] = (
            np.diff(self._magn(cdf_r_lambda(self._rh[self._ih]))) 
            / 
            np.diff(self._magn(self._rh[self._ih]))
        )
                
    @property
    def pdf(self):
        return self._psi[self._n+1][self._i]

    @property
    def r(self):
        return self._r[self._i]
    
        
    def step(self, drdt_r_lambda):        
        # for convenience (TODO!)
        G, Gh, psi, i, ih, nm, flx = self._G, self._Gh, self._psi, self._i, self._ih, self._nm, self._flux
                
        # MPDATA iterations
        for it in range(self.opts["n_it"]):
            # swap time levels
            self._n = (self._n+2) % 2 - 1
            n = self._n
            
            # boundary cond. for psi
            psi[n][:i.start] = 0
            psi[n][i.stop:] = 0
            
            # evaluate velocities
            if it == 0:
                # C = drdt * dxdr * dt / dx
                # G = 1 / dxdr
                C = self._magn(drdt_r_lambda(self._rh[ih])) / Gh[ih] * self._dt / self._dx
                self._GCh[ih] = Gh[ih] * C
                GCh = self._GCh
            else:
                self._GCh_tmp[ih] = nm.GC_antidiff(self.opts, psi[n], GCh, G, ih) 
                GCh = self._GCh_tmp             
    
            # boundary condition for GCh
            GCh[:ih.start] = 0
            GCh[ih.stop:] = 0
            
            # check CFL
            testing.assert_array_less(np.amax(GCh[ih]/Gh[ih]), 1)
            
            # integration
            if it == 0 or not self.opts["iga"]:
                flx[ih] = nm.flux(psi[n], GCh, ih)                
            else:
                flx[ih] = GCh[ih]
                
            nm.upwind(psi, flx, G, n, i)
        
            # check positive definiteness
            if self.opts["n_it"] == 1 or not self.opts["iga"]:
                assert np.amin(psi[n+1][i]) >= 0
        
            # check conservativeness (including outflow from the domain)
            bcflux = (
                max(0, GCh[(i+nm.hlf).stop-1]) * psi[n][i.stop-1] -
                min(0, GCh[(i-nm.hlf).start ]) * psi[n][i.start ] 
            ) 
            if self.opts["n_it"] == 1 or not self.opts["iga"]: # TODO TEMPORARY !!!!!!
                testing.assert_approx_equal(
                    desired = np.sum(G[i]*psi[n][i]), 
                    actual  = np.sum(G[i]*psi[n+1][i]) + bcflux,
                    significant = 15
                ) 
            